# Imaging Supernova

In this notebook, we will image a distant supernova event around the date of maximum magnitude. To fetch data, we will use the `astroquery` python package, particularly the CADC module, which queries data provided by the [Canadian Astronomical Data Centre](http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca ).

---

To launch this notebook interactively, click the Binder button. 
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opencadc/notebook-tutorials/master?filepath=%2Fastroquery_example_supernova.ipynb)

**Warning:** Binder instances have limited computational resources. A notebook will shutdown after 10 minutes of inactivity and has a maximum of 2 GB of RAM. Additionally, each repository has a maximum of 100 simultaneous users.

---


## Table of Contents
* [1. Introduction](#1.-Introduction)
* [2. Setup](#2.-Setup)
    * [2.1 Using pip](#2.1-Using-pip)
    * [2.2 From source](#2.2-From-source)
* [3. Querying](#3.-Querying)
    * [3.1 Find suitable targets](#3.1-Find-suitable-targets)
    * [3.2 Using ADQL](#3.2-Using-ADQL) 
* [4. Fetching Data](#4.-Fetching-Data)
    * [4.1 Retrieving Data with astropy.io](#4.1-Retrieving-Data-with-astropy.io)
* [5. Plotting Results](#5.-Plotting-Results)


## 1. Introduction

The [Open Supernova Catalog](https://sne.space/) is an open data collection of supernova and their parameters, such as position, redshifts, and light-curve shape. This tutorial will query the catalog for supernova with a redshift between 0.2 and 0.3. After selecting a supernova target, we will query the CADC database for the target, then display some images.

## 2. Setup
This tutorial will go through some of the basic functionalities of the CADC module in the astroquery package. The module can be installed in two ways:

### 2.1 Using pip
The CADC module is only available with the 'bleeding edge' master version of the astroquery module, and can be installed using the command:

```
    pip install https://github.com/astropy/astroquery/archive/master.zip
```

### 2.2 From source
Alternatively, you can clone and install from the source:
```
    # If you have a github account:
    git clone git@github.com:astropy/astroquery.git
    # If you do not:
    git clone https://github.com/astropy/astroquery.git
    cd astroquery
    python setup.py install
```
Note that these commands can also be done in a Jupyter notebook by either declaring the code cell a bash cell by pasting `%%bash` at the top of the cell, or preceding each line with a `!`. More information about astroquery can be found at the [astroquery github repository](https://github.com/astropy/astroquery). 


## 3. Querying

### 3.1 Find suitable targets
For this tutorial, we want to display type Ia supernova and their host galaxy in the CFHT Legacy Survey field. To find a suitable supernova, we can query the [Open Supernova Catalog](https://sne.space/) then refine the results to get a list of supernova with `claimedtype` of `Ia` and `redshift` between `0.2` and `0.3` that are part of the Supernova Legacy Survey (SNLS) where the `discoverdate` is before the `maxdate` so we can see the supernova before maximum light. The [Open Supernova Catalog API](https://github.com/astrocatalogs/OACAPI) was used to build the catalog query.



In [26]:
import os
from urllib.parse import urlencode 
import pandas as pd 
from IPython.display import Image, display, HTML
import ipywidgets as widgets 
from ipywidgets import interact, interact_manual 
from chart_studio.plotly import iplot 
from astropy import units as u 
from astropy.coordinates import SkyCoord
from astropy.time import Time, TimeDelta

params = {'claimedtype' : 'Ia', 'format' : 'csv' }
Catalog_domain ='https://api.sne.space/catalog'
query_str  = urlencode(params)
url = '{}?{}&first'.format(Catalog_domain,query_str)
data = pd.read_csv(url)

@interact
def filter_data(z_min =0.2, z_max =0.3, event =['SNLS'] ):
    global sn_data
    sn_data = data.copy()
    sn_data = sn_data.dropna(subset = ['discoverdate', 'maxdate','redshift'])
    sn_data = sn_data[(sn_data['redshift']>z_min) & (sn_data['redshift']<z_max)] 
    sn_data = sn_data[sn_data['event'].str.startswith(event)]
    sn_data = sn_data[sn_data['discoverdate']< sn_data['maxdate']]
    print('Query url: {}'.format(url))
    print('Number of results: {}'.format(len(sn_data)))
    return sn_data 

#interact(filter_data, z_min =0.2, z_max =0.3, event =['SNLS'])

interactive(children=(FloatSlider(value=0.2, description='z_min', max=0.6000000000000001, min=-0.2), FloatSlid…

Now that we have a list of suitable supernovae, we can get the area and date range that we need to query the CADC database for. Let's choose the first result from the sn_data table and extract it's R.A., Dec., and date of the maximum observation. We will also grab the "supernova" name for plotting later on. Using the R.A. and Dec., we can construct an astropy SkyCoord object, and using the maximum observation date, we can create a date window starting 20 days before and ending 100 days after to query inside of.

In [29]:
@interact
def select_supernova(supernova = sn_data['name'].tolist()):
    global start_date, end_date, coords
    selected_superNova = sn_data.loc[sn_data.name == supernova].squeeze()
    sn_name = selected_superNova['name']
    sn_ra = selected_superNova['ra']
    sn_dec =  selected_superNova['dec']
    sn_maxdate = selected_superNova['maxdate']

    coords = SkyCoord(sn_ra, sn_dec, unit=(u.hourangle, u.deg))
    days_before = 20.0
    days_after = 100.0 
    maxdate = Time(sn_maxdate.replace('/','-'), format='isot')
    start_date = maxdate - TimeDelta(days_before, format='jd')
    end_date = maxdate + TimeDelta(days_after, format='jd')

    print('Supernova coordinates: {}'.format(coords))
    print('Supernova max date: {}'.format(maxdate))
    print('Query date range: {} to {}'.format(start_date, end_date))
    
    

interactive(children=(Dropdown(description='supernova', options=('SNLS-03D4cj', 'SNLS-04D1dc', 'SNLS-05D1by', …

### 3.2 Using ADQL
The astroquery CADC python module has a `exec_sync` function which allows you to query the CADC archive database using ADQL. The query syntax requires a string and will execute the query in a batch job, which once finished, will contain the query output results. In this tutorial, we want to query images that:
   * contain the supernova coordinates
   * have been taken in between the `start_date` and `end_date` calculated above
   * have the `r` band filter
   * have been collected by the Canada France Hawaii Telescope (CFHT)
   * have calibration level 2 or above
   * have `quality_flag` that is not junk

In addition to these results, we also want an image taken much after the supernova event has occurred in order to give a good view of the host galaxy. So similarly, we will query the same coordinates, `r` filter, CFHT collection, etc, but the time the image is taken will be as new as possible.

Once we fetch the results, we will display a subset of the columns (since there are so many!). First, let's start with the query.

In [30]:
from astroquery.cadc import Cadc 
from datetime import datetime

@interact 
def using_adql(Num = ['','TOP 1'],Collection = ['CFHT', 'CFHTMEGAPIPE'], Order = ['ASC','DESC'], now = False):
    
    query_outline = """SELECT {num} * FROM caom2.Plane AS Plane 
    JOIN caom2.Observation AS Observation ON Plane.obsID = Observation.obsID
    WHERE (INTERSECTS( INTERVAL( {mjd_start}, {mjd_end} ), Plane.time_bounds_samples ) = 1 
    AND CONTAINS( POINT('ICRS', {ra}, {dec}), Plane.position_bounds ) = 1 
    AND LOWER(Plane.energy_bandpassName) LIKE '{filter}%' 
    AND collection = '{collection}'
    AND calibrationLevel >= {cal_level}
    AND (Plane.quality_flag IS NULL OR Plane.quality_flag != 'junk'))
    ORDER BY time_bounds_lower {order}"""
    
    query_params = {
    'num': Num,  # Restricts the number of results (empty string returns all)
    'mjd_start': start_date.mjd,
    'mjd_end': Time(datetime.now(), format='datetime').mjd if now else end_date.mjd,
    'ra': coords.ra.degree,
    'dec': coords.dec.degree,
    'filter': 'r',
    'collection': Collection,
    'cal_level': 2,
    'order': Order  # Order the results from oldest to newest        
    }
    
    cadc = Cadc() 
    results = cadc.exec_sync(query_outline.format(**query_params))
    columns_subset = ['productID', 'collection', 'energy_bandpassName', 'time_bounds_samples','time_bounds_lower', 'time_exposure']
    print('Total number of results: {}'.format(len(results)))

    # Showing a sample of the results
    return results[columns_subset][0:5]
    
    
    

interactive(children=(Dropdown(description='Num', options=('', 'TOP 1'), value=''), Dropdown(description='Coll…